# Exercice 2 : Sentiment Analysis on IMDB Reviews using LSTM and Keras
created by Hans Michael
<hr>

### Steps
<ol type="1">
    <li>Load the dataset (50K IMDB Movie Review)</li>
    <li>Clean Dataset</li>
    <li>Encode Sentiments</li>
    <li>Split Dataset</li>
    <li>Tokenize and Pad/Truncate Reviews</li>
    <li>Build Architecture/Model</li>
    <li>Train and Test</li>
</ol>

<hr>
<i>Import all the libraries needed</i>

In [50]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
import nltk
nltk.download('stopwords')  # to download stopwords
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thome\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<hr>
<i>Preview dataset</i>

In [51]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


<hr>
<b>Stop Word</b> is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)

<i>Declaring the english stop words</i>

In [52]:
english_stops = set(stopwords.words('english'))

<hr>

### Load and Clean Dataset

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in <b>load_dataset()</b> function, beside loading the dataset using <b>pandas</b>, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

### Encode Sentiments
In the same function, I also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.

In [53]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<hr>

### Split Dataset
In this work, I decided to split the data into 80% of Training and 20% of Testing set using <b>train_test_split</b> method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.

In [54]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
32564    [before, last, call, w, carson, daly, local, n...
19142    [as, i, peruse, hundreds, comments, loyal, rea...
23163    [course, i, wrong, now, i, never, expected, li...
22749    [this, fabulous, film, the, plot, good, yarn, ...
46651    [make, a, you, sacks, of, german, staples, wel...
                               ...                        
38762    [remember, two, stories, fondly, first, set, d...
7478     [do, not, watch, this, sad, excuse, for, a, fi...
34967    [matthew, aldrich, this, name, worthy, remembr...
40832    [i, first, came, across, my, tutor, friend, ac...
1168     [i, love, movie, the, cast, terrific, portraya...
Name: review, Length: 40000, dtype: object 

22041    [this, could, well, worst, film, i, ever, seen...
33422    [normally, would, say, loved, movie, acting, a...
46360    [steven, vasquez, directed, co, wrote, james, ...
3680     [you, sit, half, hour, watch, story, believing...
49776    [first, things, first, i, means, picky, movie,...
 

<hr>
<i>Function for getting the maximum review length, by calculating the mean of all the reviews length (using <b>numpy.mean</b>)</i>

In [55]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

<hr>

### Tokenize and Pad/Truncate Reviews
A Neural Network only accepts numeric data, so we need to encode the reviews. I use <b>tensorflow.keras.preprocessing.text.Tokenizer</b> to encode the reviews into integers, where each unique word is automatically indexed (using <b>fit_on_texts</b> method) based on <b>x_train</b>. <br>
<b>x_train</b> and <b>x_test</b> is converted into integers using <b>texts_to_sequences</b> method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using <b>tensorflow.keras.preprocessing.sequence.pad_sequences</b>.


<b>post</b>, pad or truncate the words in the back of a sentence<br>
<b>pre</b>, pad or truncate the words in front of a sentence

In [56]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 1606   142   557 ...     0     0     0]
 [  108     1 39956 ...   884     3   512]
 [  169     1   264 ...   581  9627   489]
 ...
 [ 3676 31442     8 ...     0     0     0]
 [    1    23   287 ... 13117 18170  1715]
 [    1    41     3 ...     0     0     0]] 

Encoded X Test
 [[    8    27    16 ... 79978   238 11685]
 [ 1800    12    58 ...     0     0     0]
 [ 2035 73395   427 ... 17006 10962  7311]
 ...
 [  303  5003   116 ...     0     0     0]
 [    1    31   611 ...   488  1912     7]
 [  700  5421     4 ...     0     0     0]] 

Maximum review length:  130


<hr>

### Build Architecture/Model
<b>Embedding Layer</b>: in simple terms, it creates word vectors of each word in the <i>word_index</i> and group words that are related or have similar meaning by analyzing other words around them.

<b>LSTM Layer</b>: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.
<ul>
    <li><b>Forget Gate</b>, decides information is to be kept or thrown away</li>
    <li><b>Input Gate</b>, updates cell state by passing previous output and current input into sigmoid activation function</li>
    <li><b>Cell State</b>, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.</li>
    <li><b>Ouput Gate</b>, decides the next hidden state and used for predictions</li>
</ul>

<b>Dense Layer</b>: compute the input with the weight matrix and bias (optional), and using an activation function. I use <b>Sigmoid</b> activation function for this work because the output is only 0 or 1.

The optimizer is <b>Adam</b> and the loss function is <b>Binary Crossentropy</b> because again the output is only 0 and 1, which is a binary number.

### Edit Model

I tried to reduce the overfitting of the training model. It was 0.97 accuracy for training and 0.86 accuracy for testing. I reduced the number of layer EMBED_DIM and LSTM_OUT from 64, 32 to 2, 2 so the accuracy for training is now 0.90 and for testing is 0.86. 

In [57]:
# ARCHITECTURE
EMBED_DIM = 2
LSTM_OUT = 2

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 130, 2)            184720    
                                                                 
 lstm_5 (LSTM)               (None, 2)                 40        
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 184763 (721.73 KB)
Trainable params: 184763 (721.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


<hr>

### Training
For training, it is simple. We only need to fit our <b>x_train</b> (input) and <b>y_train</b> (output/label) data. For this training, I use a mini-batch learning method with a <b>batch_size</b> of <i>128</i> and <i>5</i> <b>epochs</b>.

Also, I added a callback called **checkpoint** to save the model locally for every epoch if its accuracy improved from the previous epoch.

In [58]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [59]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5


313/313 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5061
Epoch 1: accuracy improved from -inf to 0.50612, saving model to models\LSTM.h5
313/313 [==============================] - 25s 58ms/step - loss: 0.6929 - accuracy: 0.5061
Epoch 2/5
  2/313 [..............................] - ETA: 20s - loss: 0.6918 - accuracy: 0.5273

c:\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.5815 - accuracy: 0.7037
Epoch 2: accuracy improved from 0.50612 to 0.70372, saving model to models\LSTM.h5
313/313 [==============================] - 19s 61ms/step - loss: 0.5815 - accuracy: 0.7037
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.4248 - accuracy: 0.8455
Epoch 3: accuracy improved from 0.70372 to 0.84548, saving model to models\LSTM.h5
313/313 [==============================] - 20s 63ms/step - loss: 0.4248 - accuracy: 0.8455
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.3508 - accuracy: 0.8835
Epoch 4: accuracy improved from 0.84548 to 0.88355, saving model to models\LSTM.h5
313/313 [==============================] - 19s 61ms/step - loss: 0.3508 - accuracy: 0.8835
Epoch 5/5
313/313 [==============================] - ETA: 0s - loss: 0.2966 - accuracy: 0.9073
Epoch 5: accuracy improved from 0.88355 to 0.90728, saving model to models\LSTM.h5
313/313 [================

<hr>

### Testing
To evaluate the model, we need to predict the sentiment using our <b>x_test</b> data and comparing the predictions with <b>y_test</b> (expected output) data. Then, we calculate the accuracy of the model by dividing numbers of correct prediction with the total data. Resulted an accuracy of <b>86.63%</b>

In [60]:
y_pred = model.predict(x_test, batch_size = 128)
y_pred = (y_pred > 0.5).astype('int') 

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 [==============================] - 6s 27ms/step
Correct Prediction: 8588
Wrong Prediction: 1412
Accuracy: 85.88


---

### Load Saved Model

Load saved model and use it to predict a movie review statement's sentiment (positive or negative).

In [61]:
loaded_model = load_model('models/LSTM.h5')

Receives a review as an input to be predicted

In [62]:
review = str(input('Movie Review: '))

The input must be pre processed before it is passed to the model to be predicted

In [63]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  In summary writing a movie review involves a careful balance of plot summary thesis development opinion sharing and thorough analysis of various aspects of the film including characters cinematography directorial choices music and sound and technical elements
Filtered:  ['in summary writing movie review involves careful balance plot summary thesis development opinion sharing thorough analysis various aspects film including characters cinematography directorial choices music sound technical elements']


Once again, we need to tokenize and encode the words. I use the tokenizer which was previously declared because we want to encode the words based on words that are known by the model.

In [64]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[   50  2515   397     3   610  2165  4193  2859    40  2515 13366   856
    560  5725 12288  4959   887  1320     4   485    30   523  3721  2567
    113   355  1773   715     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0]]


This is the result of the prediction which shows the **confidence score** of the review statement.

In [65]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 1s 1s/step
[[0.8879976]]


If the confidence score is close to 0, then the statement is **negative**. On the other hand, if the confidence score is close to 1, then the statement is **positive**. I use a threshold of **0.7** to determine which confidence score is positive and negative, so if it is equal or greater than 0.7, it is **positive** and if it is less than 0.7, it is **negative**

In [66]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
